# Dice with optionality

Dice game with following critere: <br>
- Player can roll *num_of_rounds* times
- Each time the player can choose to stop and claim the proceed equaled to current result, or choose to roll another time, but for the final round, he need to take proceed equaled to final result
- the dice has *n* side, with figures from 1, 2, ..., n

In [1]:
import numpy as np

n = 6 # dice with 6 facets
dice = [i+1 for i in range(n)]

# Solution
As an American type optionality is involved in such game, we need to have a decision rule to choose roll or stop. And the rule is simple if the expected return of proceeding is larger than current result, we proceed, otherwise, we stop.
<br><br>
Therefore, the program will be divided into two parts:
- generating random rolling result
- price the random result with preset decision rule
<br><br>

We use 3 kinds of random number generator:
- Fair Dice: the result will be uniformly distributed between 1,2, ..., n
- Strict Bias: the distribution of dice is biased to a certain figure i with high probability *h_prob*
- Random Bias: the distribution itself is generated by uniformly distributed random variable

## Fair Dice

The answer is easy and simple, here we just run and test the program <br>
Explanation: https://mp.weixin.qq.com/s/LBxx9w9u4yIlKtPKG_jilg

In [2]:
def rand_generator1(dice, path, num_of_rounds):
    # uniform dist
    return np.random.choice(dice, size=(path, num_of_rounds))
    
def price(decision, rands):
    
    path, num_of_rounds = rands.shape
    
    # pricing
    survive = np.ones(path, dtype=bool) # indicator for product survival
    ans = 0
    for i in range(num_of_rounds-1):
        ind = rands[:,i] >= decision[i] # indicator to take current result
        ans += np.where(ind & survive, rands[:,i], 0)
        survive &= np.where(ind, False, True)
    ans += np.where(survive, rands[:,num_of_rounds-1], 0)

    return np.mean(ans)

In [3]:
%%time
decision = []
rands = rand_generator1(dice, 100000, 1)
price(decision, rands)

Wall time: 2.99 ms


3.50499

In [4]:
decision = [3.5] # plug in result in first round
rands = rand_generator1(dice, 100000, 2)
price(decision, rands)

4.24076

In [5]:
decision = [4.25, 3.5] # plug in result in second round
rands = rand_generator1(dice, 100000, 3)
price(decision, rands)

4.65812

## Strict Bias

It seems strict bias will decrease the price of such game

In [6]:
def rand_generator2(dice, path, num_of_rounds, h_prob):
    # uniformly assign a high prob to one, while other share same prob
    dice_size = len(dice)
    rands = np.zeros((path, num_of_rounds))
    for i in range(path):
        local_high = np.random.choice(dice)
        local_dist = np.ones(dice_size) * (1 - h_prob) / (dice_size - 1)
        local_dist[local_high-1] = h_prob
        rands[i,:] = np.random.choice(dice, size=(num_of_rounds), p=local_dist)
    return rands

In [7]:
%%time
decision = []
rands = rand_generator2(dice, 100000, 1, 0.8)
price(decision, rands)

Wall time: 5.3 s


3.5069

In [8]:
decision = [3.5]
rands = rand_generator2(dice, 100000, 2, 0.8)
price(decision, rands)

3.82166

In [9]:
decision = [3.82, 3.5]
rands = rand_generator2(dice, 100000, 3, 0.8)
price(decision, rands)

3.97268

## Random Bias

While random bias will increase the price

In [10]:
def rand_generator3(dice, path, num_of_rounds):
    # dist constructed by uniformly dist variable
    dice_size = len(dice)
    rands = np.zeros((path, num_of_rounds))
    for i in range(path):
        local_dist = np.random.rand(dice_size)
        local_dist = local_dist / np.sum(local_dist)
        rands[i,:] = np.random.choice(dice, size=(num_of_rounds), p=local_dist)
    return rands

In [11]:
%%time
decision = []
rands = rand_generator3(dice, 100000, 1)
price(decision, rands)

Wall time: 4.93 s


3.49744

In [12]:
decision = [3.5]
rands = rand_generator3(dice, 100000, 2)
price(decision, rands)

4.20208

In [13]:
decision = [4.19, 3.5]
rands = rand_generator3(dice, 100000, 3)
price(decision, rands)

4.59038